# Hackathon SPE 2020 | Archivo de lectura

<br>**Presentado por:** <br>
- Geoc. Juan Camilo Burgos Flórez
- Geoc. Luis Fernando Chacón 
- Geoc. Ing. Carlos Fabián Sánchez Suárez

-----

### Desarrollo

<br> Para el desarrollo de esta actividad se contaron con datos oficiales de la ANH de reportes de producción de crudo y se trabajó para los años 2018, 2019 y 2020. Adicionalmente, con archivos encriptados "Blind Test" que se tendrán en cuenta en esta actividad. <br>
<br>El archivo de lectura contiene el código que realiza las operaciones de obtención de la información, limpia los archivos, y los convierte en formato long. Posteriormente, crea una base de datos en SQLite para facilitar el posterior manejo de datos y búsqueda de información.<br> 
<br> Sobre este mismo archivo, subido el día miércoles 4 de noviembre, se realizó el procedimiento con las nuevas variables que se asocian con los datos del Blind Test <br>

-----

In [1]:
# Importación de librerías

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3

In [2]:
# URL de los datos oficiales que se van a trabajar

url_prod_2018 = "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producci%c3%b3n%20Fiscalizada%20Crudo%202018.xlsx"
url_prod_2019 = "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producci%c3%b3n%20Fiscalizada%20Crudo%202019-DIC.xlsx"
url_prod_2020 = "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documentos%20compartidos/Producci%c3%b3n%20Fiscalizada%20Crudo%202020%20Agosto.xlsx"

# Datos para los archivos de Blind Test

#Datos blindtest
url_prod_2017bt = "https://github.com/specolombiahackathon/202010/raw/main/Data_BlindTest/Producci%C3%B3n%20Fiscalizada%20Crudo%202017.xlsx"
url_prod_2018bt = "https://github.com/specolombiahackathon/202010/raw/main/Data_BlindTest/Producci%C3%B3n%20Fiscalizada%20Crudo_2018_12458.xlsx"
url_prod_2019bt = "https://github.com/specolombiahackathon/202010/raw/main/Data_BlindTest/Producci%C3%B3n%20Fiscalizada%20Crudo%20ANH%20Colombia%202019%20-%20final.xlsx"


In [5]:
# Funcion que descarga, almaceno y limpia los datos

def get_data(data, year):
    num=None
    from_url =  pd.read_excel(data,header=None)                  # Carga el archivo de excel a una variable temporal 
    for i in range(len(from_url)):                   # Encuentra los nombres de las columnas y reasigna el encabezado
        if from_url.iloc[i,0] == "Departamento":          # A partir de la palabra "Departamento" en la primera columna 
            num = i
            break
    new_df = from_url.iloc[num:].reset_index(drop=True)
    encabezado = new_df.iloc[0].str.capitalize()     # Estandariza los nombres del encabezado
    new_df = new_df.rename(columns=encabezado)
    new_df = new_df.iloc[1:].reset_index(drop=True)
    new_df = new_df.dropna(subset=['Municipio']).dropna(axis=1)
    new_df["Año"] = year                             # Crea una nueva columna con el año al que corresponden los datos

    return new_df

In [6]:
# Obtiene los archivos y almacena la información en un data frame para cada año

df_2018 = get_data (url_prod_2018, 2018)
df_2019 = get_data (url_prod_2019, 2019)
df_2020 = get_data (url_prod_2020, 2020)

# Para los archivos del Blind Test

df_2017bt = get_data (url_prod_2017bt, 2017)
df_2018bt = get_data (url_prod_2018bt, 2018)
df_2019bt = get_data (url_prod_2019bt, 2019)


df_2019bt.head()                    # Ejemplo

,Departamento,Municipio,Operadora,Contrato,Campo,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,Año
0,cf33cb8a,cf33cb8a,d5580f74,1f d2689f,1f d2689f,7897.99,6687.2,6176.65,5998.34,5838.97,6390.66,4766.24,5871.53,5179.29,5928.18,6237.9,6226.61,2019
1,cf33cb8a,cf33cb8a,d5580f74,1f d2689f,9ac1420f,134.215,125.917,123.133,131.333,141.305,139.63,87.6278,133.443,107.876,131.583,114.477,114.988,2019
2,cf33cb8a,cf33cb8a,d5580f74,1f d2689f,9b395bc9,322.629,259.132,261.056,230.249,149.472,198.027,246.365,306.491,247.691,220.331,193.601,175.016,2019
3,cf33cb8a,cf33cb8a,d5580f74,1f d2689f,254d7db5,40.1709,42.4111,38.1699,30.2641,28.5242,29.1114,14.8439,25.7511,30.0901,6.66615,0,0,2019
4,cf33cb8a,cf33cb8a,d5580f74,29ded6f4,2f614c0b,22890.9,24367.4,26000,25748.2,24888.8,25571,19071.9,24910.7,24186.7,24413,24585.5,23529,2019


----

In [10]:
# Guarda los archivos en un solo data frame para facilidad de manejo

base_datos = pd.concat([df_2018,df_2019,df_2020]).reset_index(drop=True)
base_datos = base_datos.infer_objects()


# Para el Blind Test

base_datosbt = pd.concat([df_2017bt,df_2018bt,df_2019bt]).reset_index(drop=True)
base_datosbt = base_datosbt.infer_objects()


base_datosbt.info()    # Para conocer los tipos de las variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Departamento  122 non-null    object 
 1   Municipio     122 non-null    object 
 2   Operadora     122 non-null    object 
 3   Contrato      122 non-null    object 
 4   Campo         122 non-null    object 
 5   Enero         122 non-null    float64
 6   Febrero       122 non-null    float64
 7   Marzo         122 non-null    float64
 8   Abril         122 non-null    float64
 9   Mayo          122 non-null    float64
 10  Junio         122 non-null    float64
 11  Julio         122 non-null    float64
 12  Agosto        122 non-null    float64
 13  Septiembre    122 non-null    float64
 14  Octubre       122 non-null    float64
 15  Noviembre     122 non-null    float64
 16  Diciembre     122 non-null    float64
 17  Año           122 non-null    int64  
dtypes: float64(12), int64(1), obje

In [12]:
# Funcion que transforma la data en formato LONG

def long(data):
    melted = pd.melt(data, id_vars=["Departamento","Municipio","Operadora", "Contrato", "Campo","Año"],\
                    value_vars = ["Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre",\
                                  "Octubre","Noviembre","Diciembre"],\
                    var_name = "Mes",\
                    value_name ="Produccion")
    return melted

In [13]:
# Nuevos datos en formato LONG, que será nuestra tabla de la base de datos

base_datos = long(base_datos)

#Para el blind test
base_datosbt = long(base_datosbt)

base_datosbt

,Departamento,Municipio,Operadora,Contrato,Campo,Año,Mes,Produccion
0,cf33cb8a,cf33cb8a,d5580f74,76a16657,1f d2689f,2017,Enero,12371.083713
1,cf33cb8a,cf33cb8a,d5580f74,76a16657,9ac1420f,2017,Enero,5.367156
2,cf33cb8a,cf33cb8a,d5580f74,76a16657,9b395bc9,2017,Enero,120.462831
3,cf33cb8a,cf33cb8a,d5580f74,29ded6f4,2f614c0b,2017,Enero,23208.879752
4,cf33cb8a,cf33cb8a,d5580f74,29ded6f4,043b305e,2017,Enero,4386.088856
...,...,...,...,...,...,...,...,...
1459,0fa93c9b,756c486f,2fe52430,4b05ae15,373ebdec,2019,Diciembre,1084.287554
1460,0fa93c9b,756c486f,2fe52430,4b05ae15,e32e23a1,2019,Diciembre,414.497377
1461,657b6154,a6b36c07,ffd6d24d,38c31ea1,11e586b4,2019,Diciembre,0.000000
1462,657b6154,a6b36c07,ffd6d24d,38c31ea1,11e586b4,2019,Diciembre,408.016097


In [17]:
# Creación de la base de datos en SQLite

engine = create_engine("sqlite:///HackathonSPE.db")
sqlite_connection = engine.connect()

# Crea la tabla "TablaProduccion" en la base de datos, a partir de la información de la variable con los datos en formato long
base_datos.to_sql("TablaProduccion", sqlite_connection, if_exists ="replace")     


In [18]:
# Creación de la base de datos y la tabla del Blind Test 

engine = create_engine("sqlite:///HackathonSPE_BT.db")
sqlite_connection = engine.connect()
base_datosbt.to_sql("TablaProduccionBT", sqlite_connection, if_exists ="replace") 

-----


**VERIFICACIÓN**

In [19]:
# Cargar datos a data frame a partir de la base de datos creada
# Para verificar que se haya creado correctamente la base de datos y que su contenido corresponda 


pd.options.display.float_format = '{:.2f}'.format       # Formato de dos decimales en floats

cnx = sqlite3.connect('HackathonSPE.db')
df_produccion = pd.read_sql_query("SELECT * FROM TablaProduccion", cnx, index_col = "index")

df_produccion

,Departamento,Municipio,Operadora,Contrato,Campo,Año,Mes,Produccion
index,,,,,,,,
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,OPERACION-DIRECTA ECOPETROL,AREA TECA-COCORNA,2018,Enero,1290.89
1,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,NARE SUR,2018,Enero,219.07
2,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,UNDERRIVER,2018,Enero,800.34
3,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,OPERACION-DIRECTA ECOPETROL,AREA TECA-COCORNA,2018,Enero,289.36
4,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE,2018,Enero,11363.26
...,...,...,...,...,...,...,...,...
16051,TOLIMA,CHAPARRAL,HOCOL S.A.,TOLIMA,RÍO SALDAÑA,2020,Diciembre,nan
16052,TOLIMA,ORTEGA,HOCOL S.A.,TOLDADO,TOLDADO,2020,Diciembre,nan
16053,TOLIMA,PIEDRAS,HOCOL S.A.,PULI,TOQUI TOQUI,2020,Diciembre,nan


In [20]:
# De igual manera para el Blind Test

cnx = sqlite3.connect('HackathonSPE_BT.db')
df_produccion = pd.read_sql_query("SELECT * FROM TablaProduccionBT", cnx, index_col = "index")

df_produccion

,Departamento,Municipio,Operadora,Contrato,Campo,Año,Mes,Produccion
index,,,,,,,,
0,cf33cb8a,cf33cb8a,d5580f74,76a16657,1f d2689f,2017,Enero,12371.08
1,cf33cb8a,cf33cb8a,d5580f74,76a16657,9ac1420f,2017,Enero,5.37
2,cf33cb8a,cf33cb8a,d5580f74,76a16657,9b395bc9,2017,Enero,120.46
3,cf33cb8a,cf33cb8a,d5580f74,29ded6f4,2f614c0b,2017,Enero,23208.88
4,cf33cb8a,cf33cb8a,d5580f74,29ded6f4,043b305e,2017,Enero,4386.09
...,...,...,...,...,...,...,...,...
1459,0fa93c9b,756c486f,2fe52430,4b05ae15,373ebdec,2019,Diciembre,1084.29
1460,0fa93c9b,756c486f,2fe52430,4b05ae15,e32e23a1,2019,Diciembre,414.50
1461,657b6154,a6b36c07,ffd6d24d,38c31ea1,11e586b4,2019,Diciembre,0.00
